In [731]:
import re
import csv
import nltk
import numpy as np
from nltk.corpus import wordnet
from nltk.corpus import stopwords, brown
nltk.download('stopwords')
nltk.download('brown')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dotronghiep/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     /home/dotronghiep/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/dotronghiep/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [732]:
import pandas as pd

df = pd.read_csv('/home/dotronghiep/Documents/Datasets/Social_Listening/DataLabel.csv', header=None)
df.head(10)

,0,1,2
0,1,0,impossible to stop basically means russia is n...
1,2,0,indeed. may russia soil then selves in terror ...
2,3,0,"goodnight family, thanks to everyone for your ..."
3,4,0,keep adding more sanctionsfastest way to get r...
4,5,0,russia does not get to rape ukraine in order t...
5,6,1,its already done de facto. east ukraineand cri...
6,7,1,russia takes most of sievierodonetsk city in e...
7,8,0,biden says us to send ukraine 'advanced rocket...
8,9,1,just in - biden us to send ukraine advanced ro...
9,10,0,why do you suppose i do not understand that. i...


In [733]:
X_row = df.iloc[:, 2].values
y = df.iloc[:, 1].values

In [734]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_row, y, test_size=0.2, random_state=42, stratify=y)

# Train the model

In [735]:
print(len(X_train))

4616


In [736]:
print(len(y_train))

4616


In [737]:
# lấy danh sách stopword
stop = set(stopwords.words('english'))
# stop = ()
brown_words = set(word for word in brown.words() if word.lower() not in stop)|{"biden",
                                                                               "ukraine","kiev","kyiv","zelensky", "ukrainian", "ukrainians",
                                                                               "putin","kremlin", "russia", "russian", "russians",
                                                                               "not", "no", "nor", "against", "never", "nobody", "none", "nowhere"}

lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'[a-zA-Z]+')

In [738]:
def lemma(text):
    words = tokenizer.tokenize(text)
    cleaned_words = [lemmatizer.lemmatize(word, pos='v') for word in words if word in brown_words]
    return cleaned_words


# Loại bỏ stop words từ mỗi văn bản trong danh sách X
X_cleaned = [lemma(text) for text in X_train]


In [739]:
X_cleaned[:10]

[['ukraine', 'every', 'right', 'attack', 'russia', 'point'],
 ['real', 'men', 'defend', 'country', 'people', 'respect'],
 ['well',
  'honest',
  'russian',
  'visit',
  'board',
  'quite',
  'time',
  'russia',
  'start',
  'military',
  'build',
  'ukraine',
  'border',
  'last',
  'year',
  'call',
  'journalist'],
 ['poor',
  'russian',
  'regime',
  'world',
  'say',
  'no',
  'brutal',
  'invasion',
  'play',
  'victim',
  'get',
  'ukraine'],
 ['definitely', 'arm', 'ukraine', 'teeth', 'keep', 'russia'],
 ['legitimate',
  'security',
  'concern',
  'already',
  'countries',
  'members',
  'military',
  'base',
  'nuclear',
  'capability',
  'wipe',
  'russia',
  'map',
  'ukraine',
  'make',
  'zero',
  'difference',
  'putin',
  'know',
  'join',
  'could',
  'not',
  'invade',
  'point'],
 ['ukraine',
  'shut',
  'listen',
  'russia',
  'like',
  'shut',
  'listen',
  'us',
  'argue',
  'otherwise',
  'accept',
  'kind',
  'absurd',
  'prevalent',
  'among',
  'riddle',
  'cat',

In [740]:
import torch
import gensim.downloader as api
import csv
from gensim.models.word2vec import Word2Vec

In [741]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [742]:
max_length = max(len(sentence) for sentence in X_cleaned)
max_length

36

In [743]:
model = Word2Vec(X_cleaned, 
                 vector_size=100, 
                 window=11, 
                 min_count=5, 
                 workers=8, 
                 epochs=50)

In [744]:
vocabulary = model.wv.index_to_key
print(f"Vocabulary size: {len(vocabulary)}")

Vocabulary size: 1754


In [745]:
# Tìm từ gần nhất 
similar_words = model.wv.most_similar("putin")
for word, score in similar_words:
    print(f"{word}: {score:.4f}")

monster: 0.4552
madness: 0.4175
century: 0.4027
convince: 0.3900
killers: 0.3874
weak: 0.3870
threats: 0.3753
democracies: 0.3690
heroes: 0.3632
praise: 0.3568


In [746]:
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence

def vectorize(X, y, model=model):
    X_vectors = []
    y_vectors = []
    # Chuyển từng từ trong mỗi câu thành vector biểu diễn và thêm padding
    for sentence, label in zip(X, y):
        # Tạo một danh sách để lưu trữ các vector biểu diễn của các từ trong câu
        if sentence == []:
            continue
        sentence_vectors = []
        for word in sentence:
            # Kiểm tra xem từ đó có trong từ vựng của mô hình Word2Vec hay không
            if word in model.wv:
                word_vector = model.wv[word]
                sentence_vectors.append(word_vector)
            else:
                # Nếu từ không có trong từ vựng, bạn có thể sử dụng một vector thay thế hoặc bỏ qua nó
                # Ví dụ: word_vector = np.zeros((vector_size,))
                pass

        # Thêm padding cho câu để độ dài của mỗi câu là 35
        while len(sentence_vectors) < 35:
            sentence_vectors.append(np.zeros((100,)))  # Sử dụng vector zeros cho padding

        # Chuyển danh sách các vector thành một tensor
        sentence_tensor = torch.tensor(sentence_vectors)
        # Thêm câu đã được padding vào danh sách X_vectors
        X_vectors.append(sentence_tensor)
        y_vectors.append(label)


    # Chuyển danh sách X_vectors thành một tensor
    X_padded = np.array(X_vectors)
    y_padded = np.array(y_vectors)
    return X_padded, y_padded

# In ra kích thước của X_padded
X_input, y_input = vectorize(X_cleaned, y_train)
print(X_input.shape)
print(y_input.shape)


(4613, 35, 100)
(4613,)


In [747]:
import torch
import torch.nn as nn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Khởi tạo hidden state và cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Đưa đầu vào qua LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Lấy output của lớp cuối cùng
        out = self.fc(out[:, -1, :])
        return out

# Định nghĩa các siêu tham số
input_size = 100  # Kích thước của vectơ biểu diễn từ
hidden_size = 128  # Kích thước của hidden state trong LSTM
num_layers = 2  # Số lớp LSTM xếp chồng lên nhau
num_classes = 2  # Số lớp đầu ra
num_epochs = 100  # Số epoch
lr=0.001

# Khởi tạo mô hình
lstm = LSTMClassifier(input_size, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=lr)
lstm.to(device)

X_train, X_valid, y_train, y_valid = train_test_split(X_input, y_input, test_size=0.2, random_state=42, stratify=y_input)

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_valid = torch.tensor(X_valid, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train).to(device)
y_valid = torch.tensor(y_valid).to(device)
# Huấn luyện mô hình

for epoch in range(1, num_epochs + 1):
    # Huấn luyện
    lstm.train()
    optimizer.zero_grad()
    outputs = lstm(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # In ra loss sau mỗi 5 epoch
    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

        # Đánh giá mô hình trên tập validation
        lstm.eval()
        with torch.no_grad():
            outputs = lstm(X_valid)
            _, predicted = torch.max(outputs, 1)

            accuracy = accuracy_score(y_valid.cpu(), predicted.cpu())
            print(f"Accuracy on validation set: {accuracy}")
            print("=========================================")

Epoch 5, Loss: 0.6543352007865906
Accuracy on validation set: 0.7118093174431203
Epoch 10, Loss: 0.6069053411483765
Accuracy on validation set: 0.7118093174431203
Epoch 15, Loss: 0.6005474925041199
Accuracy on validation set: 0.7118093174431203
Epoch 20, Loss: 0.6029213070869446
Accuracy on validation set: 0.7118093174431203
Epoch 25, Loss: 0.5997281074523926
Accuracy on validation set: 0.7118093174431203
Epoch 30, Loss: 0.6005759835243225
Accuracy on validation set: 0.7118093174431203
Epoch 35, Loss: 0.5993765592575073
Accuracy on validation set: 0.7118093174431203
Epoch 40, Loss: 0.599163830280304
Accuracy on validation set: 0.7118093174431203
Epoch 45, Loss: 0.5978242754936218
Accuracy on validation set: 0.7118093174431203
Epoch 50, Loss: 0.5950351357460022
Accuracy on validation set: 0.7118093174431203
Epoch 55, Loss: 0.5832815170288086
Accuracy on validation set: 0.7118093174431203
Epoch 60, Loss: 0.5734872817993164
Accuracy on validation set: 0.7085590465872156
Epoch 65, Loss: 0.